# This notebook shows an example of resume summarization into a cover letter using OpenAI's gpt-4o-mini 

In [ ]:
# This cell contains the imports for our summarization solution
import os
import requests
from dotenv import load_dotenv
from pypdf import PdfReader
from openai import OpenAI
from docx import Document

In [ ]:
# Loading environment variables in a file called .env

load_dotenv(override=True)
api_key = os.getenv('OPENAI_API_KEY')

# Check the key

if not api_key:
    print("No OpenAI API key was found!")
elif not api_key.startswith("sk-proj-"):
    print("A key was found, but it doesn't start sk-proj-; Check your key again!")
elif api_key.strip() != api_key:
    print("A key was found, but it looks like it might have space or tab characters at the start or end - please remove them!")
else:
    print("Your OpenAI API key was found!")

In [ ]:
openai = OpenAI()

In [ ]:
reader = PdfReader("Insert the name of or the path to your PDF file here.pdf")

In [ ]:
def merge_pages(current_page, next_page):
    #This function merges the text from a multipage pdf document
    current_page.merge_page(page2=next_page, over=True)
    return

In [ ]:
# This for loop iterates over the pages of your resume and merges them into one single page with all the content combined
for page_number in range(len(reader.pages)):
    print(f"Page number:{page_number}.")
    
    if page_number == 0:
        page = reader.pages[page_number]
        next_page = reader.pages[page_number+1]
        merge_pages(page, next_page)
    elif page_number > 0 and page_number < len(reader.pages)-1:
        next_page = reader.pages[page_number+1]
        merge_pages(page, next_page)
    else:
        break      

In [ ]:
# Printing the content merged into just one page
print(page.extract_text())

In [ ]:
# Extracting the text from the merged page
resume_text = page.extract_text()

In [ ]:
# Creating a class to represent a resume

class Resume:
    def __init__(self, text):
        self.text = text

In [ ]:
# Defining the system prompt

system_prompt = "You are an assistant that analyzes the content of a resume \
and provides a short summary in a cover letter for a job application.\ Respond in a cover letter format."

In [ ]:
# This function that defines and returns the user prompt for a resume

def user_prompt_for(resume):
    user_prompt = f"You are looking at a resume."
    user_prompt += "\nThe content of this resume is as follows; \
please provide a short summary of this resume in a cover letter for a job application. \
If it includes skills and work experience, then summarize these too.\n\n"
    user_prompt += resume.text
    return user_prompt

In [ ]:
def messages_for(resume):
    return [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt_for(resume)}
    ]

In [ ]:
def summarize(text):
    resume = Resume(text)
    response = openai.chat.completions.create(
        model = "gpt-4o-mini",
        messages = messages_for(resume)
    )
    return response.choices[0].message.content

In [ ]:
summary = summarize(resume_text)

In [ ]:
summary

In [ ]:
# The lines below save the summary as a MS Word file

# Creating a Word document
document = Document()

# Adding the summary to the document as a paragraph
document.add_paragraph(summary)

# Saving the document to a docx file
document.save("cover_letter.docx")

print("The file 'cover_letter.docx' was successfully saved!")